## 1. conversation

In [28]:
from collections import deque
def splitMessageToUserWidth222(message, userWidth):
    wordList = message.split(' ')
    queue, out = [], []
    while wordList:
        if len(wordList[0]) + 1 + len(queue) > userWidth:
            out.append("".join)
    pass
    
def splitMessageToUserWidth(message, userWidth):
    out = []
    wordList = message.split(' ')
    i = 0
    while i < len(wordList) - 1:
        curChunk = wordList[i]
        
        if len(curChunk) == userWidth:
            out.append(curChunk)
            i += 1
            continue
            
        for j in range(i+1, len(wordList)):
            if len(curChunk) + 1 + len(wordList[j]) == userWidth:
                curChunk += ' ' + wordList[j]
                i = j + 1
                break
                
            elif len(curChunk) + 1 + len(wordList[j]) > userWidth:
                i = j
                break
            
            else:
                curChunk += ' ' + wordList[j]
                i = j                
        
        out.append(curChunk)
    
    out.append(wordList[-1])
        
    return out

def conversation(messages, width, userWidth):
    header = "+" + "".join(["*" for _ in range(width)]) + "+"
    out = [header]
    for m in messages:
        userId, userMessage = m[0], m[1]
#         userMessageChunk = m[1].split(' ')
        userMessageChunk = splitMessageToUserWidth222(m[1], userWidth)
        if userId == "1":
            for chunk in userMessageChunk:
                out.append("|" + chunk + " " * (width - len(chunk)) + "|")
        if userId == "2":
            for chunk in userMessageChunk:
                out.append("|" + " " * (width - len(chunk)) + chunk + "|")
    out += [header]
    return out

In [29]:
messages = [["1", "Hello how r u"],
           ["2", "good ty"],
           ["2", "u"],
           ["1", "me too bro"]]
conversation(messages, 15, 5)

['+***************+',
 '|Hello          |',
 '|          good |',
 '|             ty|',
 '|              u|',
 '|me             |',
 '|too            |',
 '|bro            |',
 '+***************+']

In [26]:
list("foobar")

['f', 'o', 'o', 'b', 'a', 'r']

## 2. change directory

In [34]:
def changeDir(commands):
    stack = []
    for command in commands:
        dest = command.removeprefix('cd ')
        if dest == '.':
            continue
        if dest == '/':
            stack = []
        elif dest == '..':
            stack.pop()
        else:
            stack.append(dest)
    return "/" + "/".join(stack)

In [36]:
commands = [
    "cd users",
    "cd codesignal",
    "cd ..",
    "cd admin"
]
changeDir(commands)

'/users/admin'

## 3. group chat

In [101]:
import re
from collections import defaultdict

def sortByValueThenKey(d):
    sortedFreq = sorted(d.items(), key=lambda x: (-x[1],x[0]))
    return [s[0] + '=' + str(s[1]) for s in sortedFreq]

def trimMatch(match):
    return ''.join(c for c in match if c not in '@,')

def mentions(members, messages):
    patterns = ['\B@\w+', '\B@\w+(,\w+)*']
    usersFreq = defaultdict(lambda: 0)
    
    for m in messages:
        usersAppear = set()
        for pattern in patterns:
            matches = re.findall(pattern, m)
            for match in matches:
                if len(match) == 0:
                    continue
                match = trimMatch(match)
                usersAppear.add(match)
        # put users appeared in this message into frequency table
        for user in usersAppear:
            usersFreq[user] += 1
    
    # count only member frequency
    membersFreq = {member: 0 for member in members}
    for user, freq in usersFreq.items():
        if user not in members:
            continue
        membersFreq[user] = freq
    # sort by frequency first, then user id lexicorically
    membersFreq = sortByValueThenKey(membersFreq)
    
    return membersFreq


In [102]:
members = ["id123", "id234", "id7", "id321"]
messages = [
    "Hey @id123,id321 review this PR please! @id123 what do you think about this approach?",
    "Hey @id7 nice appro@ch! Great job! @id800 what do you think?",
    "@id123,id321 thx!"
]
# ["id123=2", "id321=2", "id7=1", "id234=0"]

In [103]:
mentions(members, messages)

['id123=2', 'id321=2', 'id7=1', 'id234=0']

## 4. Black cells

In [130]:
def blackCells(n, m, black):
    matrix = [[0 for _ in range(m)] for _ in range(n)]
    for i in range(n):
        for j in range(m):
            if [i, j] in black:
                matrix[i][j] = 1
    addSum = [[0 for _ in range(m+1)] for _ in range(n+1)]
    for i in range(1, n+1):
        addSum[i][1] = addSum[i-1][1] + matrix[i-1][0]
    for j in range(1, m+1):
        addSum[1][j] = addSum[1][j-1] + matrix[0][j-1]
    for i in range(2, n+1):
        for j in range(2, m+1):
            addSum[i][j] = addSum[i-1][j] + addSum[i][j-1] - addSum[i-1][j-1] + matrix[i-1][j-1]

    res = [0 for _ in range(5)]
    for i in range(2, n+1):
        for j in range(2, m+1):
            numOfBlack = addSum[i][j] - addSum[i-2][j] - addSum[i][j-2] + addSum[i-2][j-2]
            res[numOfBlack] += 1
    return res

In [131]:
blackCells(3, 3, [[0,0], [0,1], [1,0]])

[1, 2, 0, 1, 0]

In [132]:
blackCells(4, 4, [[0,1],[0,2], [1,0],[2,0], [2,1], [3,3]])

[1, 4, 3, 1, 0]